## Initial Setting

In [1]:
# Link to Drive
from google.colab import drive
drive.mount('/content/drive')

# Connect to Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

import os
from glob import glob

Mounted at /content/drive
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=agffNGDRtptl8cIG-vZjcTOO6xNHCCJHz7c0CIYe8Bg&tc=IpAa6N-tp-RCqTXExJhyO0rH9tvsI9tSunzfITnGMik&cc=3dVVmYzspuqLTsOVVEf_xMIk1n4V_p9ymvnWlnxVsLs

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk4sARof-tp4Xm4VMFOAEgQ8lfdUC-XDjcfEzDfXEtlU5e6uxERd7Hc

Successfully saved authorization token.


In [2]:
!pip install geemap
import geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.4 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11135 sha256=362d5e27d749bf3850f0eb955777dda6a89fd42bb1bf269e29af0

## User Input

In [3]:
# Customerize Country
country_name = "Cameroon"

## Get Admin Data

In [4]:
# Get country boundary of Madagascar
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
aoi = ee.Feature(countries.filter(ee.Filter.eq('country_na', country_name)).first())

In [5]:
bbox = aoi.geometry().bounds()
bbox.getInfo()

{'geodesic': False,
 'type': 'Polygon',
 'coordinates': [[[8.498415023331367, 1.655899997323747],
   [16.194408018154032, 1.655899997323747],
   [16.194408018154032, 13.08333622384212],
   [8.498415023331367, 13.08333622384212],
   [8.498415023331367, 1.655899997323747]]]}

## Get Tree Cover

In [6]:
img_hansen = ee.Image("UMD/hansen/global_forest_change_2021_v1_9").clip(bbox)

### Forest Cover Area in 2000

In [7]:
# Forest is defined as canopy cover >=10%, area >= 0.5ha
img_fc00 = img_hansen.select(['treecover2000']).gte(10)
img_fc00 = img_fc00.rename(['fc_2000']).set({"layer": "fc_2000"})

In [ ]:
# Treeloss Mask
#lossTill01 = lossyear.gte(1).And(lossyear.lte(21))
#img_forest01 = img_forest00.where(lossTill01.eq(1), 0).selfMask()

### Annual Forest Cover

In [8]:
# Get Image of lossyear and loss
lossyear = img_hansen.select(['lossyear'])
loss = img_hansen.select(['loss'])

In [9]:
# Filter to get the most recent year
recent = lossyear.reduceRegion(reducer = ee.Reducer.max(), 
                              geometry = bbox,
                              maxPixels = 1e12)

year_index = range(1, recent.getInfo()['lossyear']+1)
year_name = range(2001, recent.getInfo()['lossyear']+2001)

# Pair Year Index and Year Name
year_pair = dict(zip(year_index, year_name))
#year_pair

In [10]:
# Function to get Binary Image of annual forest cover 
def getAnnualForest(yearIndex):
  # Mask: loss occurred (loss = 1), cumulative loss from 2001 to a specific year
  accumuLoss_t = loss.eq(1).And(lossyear.gte(1)).And(lossyear.lte(yearIndex))
  # Mask out loss pixels
  img_fcX = img_fc00.where(accumuLoss_t.eq(1), 0)#.selfMask()
  # Name the output layer
  img_fcX = img_fcX.rename(["fc_"+str(year_pair[yearIndex])])
  img_fcX = img_fcX.set({"layer": "fc_"+str(year_pair[yearIndex])})
  return img_fcX

In [11]:
# Apply the function to get a list of binary images of annual forest cover
lst_fcX = list(map(getAnnualForest, year_index))
# Add Image "forest cover 2000" to the list
lst_fcAll = ee.List([img_fc00]).cat(lst_fcX)
print(lst_fcAll.size().getInfo())

# Turn list into an image collection
imc_fcAll = ee.ImageCollection(lst_fcAll)
#imc_fcAll.size().getInfo()

# Turn Image Collection into an Image
img_fcAll = imc_fcAll.toBands()
print(img_fcAll.bandNames().getInfo())

22
['0_fc_2000', '1_fc_2001', '2_fc_2002', '3_fc_2003', '4_fc_2004', '5_fc_2005', '6_fc_2006', '7_fc_2007', '8_fc_2008', '9_fc_2009', '10_fc_2010', '11_fc_2011', '12_fc_2012', '13_fc_2013', '14_fc_2014', '15_fc_2015', '16_fc_2016', '17_fc_2017', '18_fc_2018', '19_fc_2019', '20_fc_2020', '21_fc_2021']


In [12]:
# Visualize
cent_lon = aoi.centroid().getInfo()['geometry']['coordinates'][0]
cent_lat = aoi.centroid().getInfo()['geometry']['coordinates'][1]

Map1 = geemap.Map(center=(cent_lat, cent_lon), zoom=5)

Map1.addLayer(img_fcAll.select(1), vis_params={'min':0, 'max':1, 'palette': ['grey', 'lightgreen']}, name="fc_2000")
Map1.addLayer(img_fcAll.select(20), vis_params={'min':0, 'max':1, 'palette':['grey', 'lightgreen']}, name="fc_2019")
#Map1.addLayer(loss.pixelArea().clip(bbox), vis_params={'min':0, 'max':1000, 'palette':["#707b41","#7cdd51","#c2d39a","#ccd04a","#5fa54b"]}, name="pxArea_hansen")

Map1

Map(center=[5.683679386840686, 12.738353454423779], controls=(WidgetControl(options=['position', 'transparent_…

In [13]:
# Export Image
task_config = {
    'description': 'fc',
    'fileNamePrefix': 'fcCMR',
    'crs': 'EPSG:4326',
    'scale': 30,  
    'region': bbox.getInfo()['coordinates'],
    'folder': "paMatching_img",
    'skipEmptyTiles': True,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 10e12
    }
task1 = ee.batch.Export.image.toDrive(img_fcAll, **task_config)
task1.start() 

In [35]:
task1.status()

{'state': 'COMPLETED',
 'description': 'dem',
 'creation_timestamp_ms': 1682329544426,
 'update_timestamp_ms': 1682330034818,
 'start_timestamp_ms': 1682329560808,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/#folders/1rPxt5EiXhLtq42KzG4KEzAeIPa8IwvSM'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 137.23809814453125,
 'id': 'NI4YLSFSPURC5AFM6CFVS3JY',
 'name': 'projects/earthengine-legacy/operations/NI4YLSFSPURC5AFM6CFVS3JY'}

## Elevation

In [18]:
# Elevation
img_srtm = ee.Image("CGIAR/SRTM90_V4")

In [19]:
# Band Names
print(img_srtm.bandNames().getInfo())
# CRS
print(img_srtm.projection().getInfo())
# Spatial Resolution
print(img_srtm.projection().nominalScale().getInfo())

['elevation']
{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [0.000833333333333, 0, -180, 0, -0.000833333333333, 60]}
92.76624232769086


In [20]:
# Export Image
task_config = {
    'description': 'dem',
    'fileNamePrefix': 'srtmCMR',
    'crs': 'EPSG:4326',
    'scale': 92.77,  
    'region': bbox.getInfo()['coordinates'],
    'folder': "paMatching_img",
    'skipEmptyTiles': True,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 10e12
    }
task2 = ee.batch.Export.image.toDrive(img_srtm, **task_config)
task2.start() 

In [33]:
task2.status()

{'state': 'RUNNING',
 'description': 'dem',
 'creation_timestamp_ms': 1682329544426,
 'update_timestamp_ms': 1682329801313,
 'start_timestamp_ms': 1682329560808,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'NI4YLSFSPURC5AFM6CFVS3JY',
 'name': 'projects/earthengine-legacy/operations/NI4YLSFSPURC5AFM6CFVS3JY'}

## Travel Time

In [22]:
img_travel = ee.Image("Oxford/MAP/accessibility_to_cities_2015_v1_0")

In [23]:
# Band Names
print(img_travel.bandNames().getInfo())
# CRS
print(img_travel.projection().getInfo())
# Spatial Resolution
print(img_travel.projection().nominalScale().getInfo())

['accessibility']
{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [0.00833333, 0, -180, 0, -0.00833333, 84.999942]}
927.6620522123104


In [24]:
# Export Image
task_config = {
    'description': 'travel',
    'fileNamePrefix': 'travelTime_CMR',
    'crs': 'EPSG:4326',
    'scale': 927.67,  
    'region': bbox.getInfo()['coordinates'],
    'folder': "paMatching_img",
    'skipEmptyTiles': True,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 10e12
    }
task3 = ee.batch.Export.image.toDrive(img_travel, **task_config)
task3.start() 

In [31]:
task3.status()

{'state': 'COMPLETED',
 'description': 'travel',
 'creation_timestamp_ms': 1682329694919,
 'update_timestamp_ms': 1682329736389,
 'start_timestamp_ms': 1682329711416,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/#folders/1rPxt5EiXhLtq42KzG4KEzAeIPa8IwvSM'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 0.6947263479232788,
 'id': 'G2FIVGP3HEFNTVV4KXQEP6NE',
 'name': 'projects/earthengine-legacy/operations/G2FIVGP3HEFNTVV4KXQEP6NE'}

## Clay Content in Soil

In [27]:
img_clay = ee.Image("ISDASOIL/Africa/v1/clay_content").select(['mean_0_20'])

In [28]:
# Band Names
print(img_clay.bandNames().getInfo())
# CRS
print(img_clay.projection().getInfo())
# Spatial Resolution
print(img_clay.projection().nominalScale().getInfo())

['mean_0_20']
{'type': 'Projection', 'crs': 'EPSG:3857', 'transform': [30, 0, -3502582.8076430312, 0, -30, 4530073.13045243]}
30


In [29]:
# Export Image
task_config = {
    'description': 'clay',
    'fileNamePrefix': 'clay_CMR',
    'crs': 'EPSG:3857',
    'scale': 30,  
    'region': bbox.getInfo()['coordinates'],
    'folder': "paMatching_img",
    'skipEmptyTiles': True,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 10e12
    }
task4 = ee.batch.Export.image.toDrive(img_clay, **task_config)
task4.start() 

In [34]:
task4.status()

{'state': 'RUNNING',
 'description': 'clay',
 'creation_timestamp_ms': 1682329771973,
 'update_timestamp_ms': 1682330084583,
 'start_timestamp_ms': 1682329782927,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': '2OCSMZLYXWTD242WCDCL2MWO',
 'name': 'projects/earthengine-legacy/operations/2OCSMZLYXWTD242WCDCL2MWO'}